In [4]:
import os
import rawpy
import torch
import torchvision.transforms as T
from torchvision import models
from PIL import Image
import numpy as np



In [5]:
def enhance_with_model(image_np):
    """
    Enhance the image using a pretrained deep model.
    Optimized for Apple Silicon (MPS) or CPU fallback.
    """
    if torch.backends.mps.is_available():
        device = "mps"
    else:
        device = "cpu"

    transform_in = T.Compose([
        T.ToTensor(),
        T.Resize((224, 224)),
        T.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225])
    ])

    transform_out = T.Compose([
        T.Resize((image_np.shape[0], image_np.shape[1])),
        T.ToPILImage()
    ])

    img_tensor = transform_in(Image.fromarray(image_np)).unsqueeze(0).to(device)

    model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    model = model.to(device)
    model.eval()

    with torch.no_grad():
        features = model.conv1(img_tensor)
        mean_intensity = features.mean().item()

    adjustment = np.clip(1.2 - mean_intensity, 0.8, 1.4)
    image_np = np.clip(image_np * adjustment, 0, 255).astype(np.uint8)

    enhanced_image = transform_out(Image.fromarray(image_np))
    return enhanced_image


def convert_dng_to_jpeg_with_enhancement(source_folder, destination_folder):
    os.makedirs(destination_folder, exist_ok=True)
    dng_files = [f for f in os.listdir(source_folder) if f.lower().endswith('.dng')]

    if not dng_files:
        print("No DNG files found in the source folder.")
        return

    print(f"Found {len(dng_files)} DNG files. Starting intelligent enhancement...\n")

    for filename in dng_files:
        input_path = os.path.join(source_folder, filename)
        output_filename = os.path.splitext(filename)[0] + ".jpg"
        output_path = os.path.join(destination_folder, output_filename)

        try:
            with rawpy.imread(input_path) as raw:
                rgb = raw.postprocess(
                    use_camera_wb=True,
                    no_auto_bright=True,
                    output_bps=8,
                    gamma=(2.2, 4.5)
                )

            enhanced = enhance_with_model(rgb)
            enhanced.save(output_path, "JPEG", quality=95)
            print(f"Enhanced and saved: {output_filename}")

        except Exception as e:
            print(f"Error processing {filename}: {e}")

    print("\nAll conversions completed successfully!")




In [6]:
if __name__ == "__main__":
    source = input("Enter path to folder containing .dng files: ").strip()
    destination = input("Enter destination folder for .jpg files: ").strip()
    convert_dng_to_jpeg_with_enhancement(source, destination)

Enter path to folder containing .dng files:  /Users/adityapathak/Downloads/Clyde river at night
Enter destination folder for .jpg files:  /Users/adityapathak/Downloads/test convert dng to jpg


Found 10 DNG files. Starting intelligent enhancement...

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/adityapathak/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
Error processing IMG_2209.dng: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)>
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/adityapathak/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
Error processing IMG_2218.dng: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)>
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/adityapathak/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
Error processing IMG_2217.dng: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)>
Downloading: "https://